# Background 
This code implements a spiking neural net with conductance in input. the following equations govern the dynamic of the network. 
### transmembrane voltage dynamics
first we model the transmembrane voltage as 
$$\tau_m \frac{dV_i}{dt}= - V_i(t)+ R_m \times I^{syn}_i(t) $$ 
$$ {\tau_a}_i \frac{dB_i(t)}{dt} = b_i^0 -B_i(t)$$ 
where, $R_m$ is membrane resistance, $\tau_m$ is membrane time constant, and ${\tau_a}_i$ is adaptation time constant  .
the synaptic current relates to synaptic activations in the following way
$$I^{syn}_i(t)= \sum_j W^{in}_{ij} \times X(t) + \sum_j W^{rec}_{ij} \times S_j(t) $$ 

### neuron firing dynamics 
The firing dynamics of the neuron is model as a simple reseting. More specifically, 
$$V_i \rightarrow V_{reset} \ \ \  if \ \ \ V_i>=B_{i} $$

$ V_{\Theta}$ represent the threshold voltage and $V_{reset}$ is the reset voltage of the neuron.

### Input dynamics 
Input synapes are the the site of learning in the spiking network. Below a conductance based formulation is presented. 
First, the time-dependent input conductance to membrane is calculated as follows 
$$ g_i(t) = \sum_j W_{ij} S_{j}(t) $$

in the current version $S_{j}(t)$ is equal to spike at timestep $t$ without any decay dynamics. 
-  TODO the term $j$ reperesent all the neurons that have a synapse onto the neuron $i$. the time dependence of conductance is due to $S(t)$ which represent the spiking activity for neurons connected to neuron $i$ . The spiking activity has the following governing equations 
$$ S_{j} \rightarrow S_{j}+1 \quad if \ neuron\ j\ fires$$
$$ \frac{dS_{j}(t)}{dt} = \frac{-S_{j}(t)}{\tau_s}$$ 

### Spike Adaptation dynamics 
The threshold for spiking increases with every spike emited from a neuron with the following dynamics 
$$ B_{i}(t) \rightarrow B_{i}(t)+\frac{\beta}{{\tau_a}_i} \quad if \ neuron\ i\ fires$$

### References 
-  Fiete, Ila R., Walter Senn, Claude Z. H. Wang, and Richard H. R. Hahnloser. 2010. “Spike-Time-Dependent Plasticity and Heterosynaptic Competition Organize Networks to Produce Long Scale-Free Sequences of Neural Activity.” Neuron 65 (4): 563–76. 

-  Bellec, Guillaume, Darjan Salaj, Anand Subramoney, Robert Legenstein, and Wolfgang Maass. 2018. “Long Short-Term Memory and Learning-to-Learn in Networks of Spiking Neurons.” arXiv [cs.NE]. arXiv. http://arxiv.org/abs/1803.09574.



### notes about the implementation 
-  both input and recurrent weight are combined into 1 kernel matrix 


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt 
%matplotlib inline
import matplotlib.cm as cm

import numpy as np
import tensorflow as tf

import collections
import hashlib
import numbers
from datetime import datetime
from pathlib import Path
import os

from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.layers import base as base_layer
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import partitioned_variables
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import variables as tf_variables
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import nest
from tensorflow.contrib.rnn.python.ops.core_rnn_cell import _Linear
from tensorflow.contrib import slim
from matplotlib.colors import LinearSegmentedColormap

import kernl_spiking_cell_v4 as kernl_spiking_cell


first testing the code on a constant input 

In [ ]:
batch_size = 1
sequence_length = 50
num_inputs=1
num_units=2
#input_spikes=np.random.randint(2,size=[batch_size,sequence_length,num_inputs])
input_spikes=1.0*np.ones([batch_size,sequence_length,num_inputs])
tf.reset_default_graph()

tf_input_values = tf.constant(input_spikes, dtype=tf.float32)
kernl_snn = kernl_spiking_cell.kernl_spike_Cell(num_units=num_units,
                                                num_inputs=num_inputs,
                                                state_is_tuple=True,
                                                output_is_tuple=True,
                                                tau_s=10.0,
                                                tau_refract=5.0,
                                                tau_m=20.0,
                                                noise_param=1.0,
                                                time_steps=sequence_length,
                                                kernel_initializer=tf.initializers.random_uniform(),
                                                bias_initializer=tf.initializers.zeros())
outputs, state = tf.nn.dynamic_rnn(cell=kernl_snn, dtype=tf.float32, inputs=tf_input_values)

cell_outputs=[]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run , state_run = sess.run([outputs, state])

    variables_names =[v.name for v in tf.global_variables()]
    values = sess.run(variables_names)
    for k,v in zip(variables_names, values):
        print(k, v) 
plt.figure(figsize=[15,15])
ax=plt.subplot(5,1,1)
ax.plot(output_run.v_mem.squeeze(axis=0))
ax.set_title('v_mem')
ax=plt.subplot(5,1,2)
ax.plot(output_run.v_mem_hat.squeeze(axis=0),'--')
ax.set_title('v_mem_hat')
ax=plt.subplot(5,1,3)
ax.plot(output_run.psi.squeeze(axis=0))
ax.set_title('noise')
ax=plt.subplot(5,1,4)
ax.plot(output_run.v_mem.squeeze(axis=0)-output_run.v_mem_hat.squeeze(axis=0))
ax.set_title('h-h_hat')
ax=plt.subplot(5,1,5)
ax.plot(np.cumsum(output_run.v_mem.squeeze(axis=0)-output_run.v_mem_hat.squeeze(axis=0),axis=0))
ax.set_title('integral of (h-h_hat)')


next we create a recurrent version to verify the functionality 

### computation graph for learning $M$ and $K(\tau)$

In the original version of kernl implemented in rnn, the value of h was a continues function in time, and the perturbation could accumulate throughout a trial, which made the calculation of difference between perturbed and non-perturbed h straightforward. However in the spiking network here, there is a complication with the value of v_mem being discontinuous. how can we aleviate this problem. based on the example about, we can see that if we integrate the difference between them, we get a smoother function to calculate gradients from. we first try to optimize based on final difference between the integrals.  

In [8]:
batch_size = 1
sequence_length = 50
num_inputs=1
num_units=2
#input_spikes=np.random.randint(2,size=[batch_size,sequence_length,num_inputs])
input_spikes=1.0*np.ones([batch_size,sequence_length,num_inputs])

tf.reset_default_graph()
graph=tf.Graph()
with graph.as_default():
    # define the network 
    tf_input_values = tf.constant(input_spikes, dtype=tf.float32)
    kernl_snn = kernl_spiking_cell.kernl_spike_Cell(num_units=num_units,
                                                num_inputs=num_inputs,
                                                state_is_tuple=True,
                                                output_is_tuple=True,
                                                tau_s=10.0,
                                                tau_refract=5.0,
                                                tau_m=20.0,
                                                noise_param=1.0,
                                                time_steps=sequence_length,
                                                kernel_initializer=tf.initializers.random_uniform(),
                                                bias_initializer=tf.initializers.zeros())
    kernl_outputs, kernl_state = tf.nn.dynamic_rnn(cell=kernl_snn, dtype=tf.float32, inputs=tf_input_values)
    trainables=tf.trainable_variables()
    variable_names=[v.name for v in tf.trainable_variables()]
    # 
    find_joing_index = lambda x, name_1,name_2 : [a and b for a,b in zip([np.unicode_.find(k.name, name_1)>-1 for k in x] ,[np.unicode_.find(k.name, name_2)>-1 for k in x])].index(True)
    # find trainable parameters for kernl 
    with tf.name_scope('kernl_Trainables') as scope:
        kernl_temporal_filter_index= find_joing_index(trainables,'kernl','temporal_filter')
        kernl_sensitivity_tensor_index= find_joing_index(trainables,'kernl','sensitivity_tensor')
    # 
        kernl_tensor_training_indices=np.asarray([kernl_sensitivity_tensor_index,kernl_temporal_filter_index],dtype=np.int)
        kernl_tensor_trainables= [trainables[k] for k in kernl_tensor_training_indices]
     
    with tf.name_scope('kernl_train_tensors') as scope: 
        state_diff_int=tf.reduce_sum(tf.subtract(kernl_outputs.v_mem_hat[:,:,:], kernl_outputs.v_mem[:,:,:]),axis=1)
        #estimate_state_diff_int=tf.matmul(kernl_hidden_states.Theta[:,,:],trainables[kernl_sensitivity_tensor_index])
        #kernl_loss_state_prediction=tf.losses.mean_squared_error(tf.subtract(kernl_hidden_states.v_mem_hat[:,-1,:], kernl_hidden_states.v_mem[:,-1,:]),tf.matmul(kernl_hidden_states.Theta[:,-1,:],trainables[kernl_sensitivity_tensor_index]))
        #kernl_tensor_optimizer = tf.train.RMSPropOptimizer(learning_rate=tensor_learning_rate)
        #kernl_tensor_grads=tf.gradients(ys=kernl_loss_state_prediction,xs=kernl_tensor_trainables)
        #kernl_tensor_grad_and_vars=list(zip(kernl_tensor_grads,kernl_tensor_trainables))
        #kernl_tensor_train_op=kernl_tensor_optimizer.apply_gradients(kernl_tensor_grad_and_vars)
        
    
    ##################
    # SUMMARIES ######
    ##################
    
    #with tf.name_scope("kernl_tensor_summaries") as scope: 
        # kernl sensitivity tensor 
     #   tf.summary.histogram('kernl_sensitivity_tensor_grad',kernl_tensor_grads[0]+1e-10)
     #   tf.summary.histogram('kernl_sensitivity_tensor',trainables[kernl_sensitivity_tensor_index]+1e-10)
        # kernl temporal filter 
     #   tf.summary.histogram('kernl_temporal_filter_grad',kernl_tensor_grads[1]+1e-10)
     #   tf.summary.histogram('kernl_temporal_filter',trainables[kernl_temporal_filter_index]+1e-10)
        # kernl loss 
     #   tf.summary.scalar('kernl_loss_state_prediction',kernl_loss_state_prediction+1e-10)
        # kernl senstivity tensor and temporal filter 
     #   tf.summary.image('kernl_sensitivity_tensor',tf.expand_dims(tf.expand_dims(trainables[kernl_sensitivity_tensor_index],axis=0),axis=-1))
     #   tf.summary.image('kernl_sensitivity_tensor_grad',tf.expand_dims(tf.expand_dims(kernl_tensor_grads[0],axis=0),axis=-1))
     #   tf.summary.image('kernl_temporal_filter',tf.expand_dims(tf.expand_dims(tf.expand_dims(trainables[kernl_temporal_filter_index],axis=0),axis=-1),axis=-1))
     #   tf.summary.image('kernl_temporal_filter_grad',tf.expand_dims(tf.expand_dims(tf.expand_dims(kernl_tensor_grads[1],axis=0),axis=-1),axis=-1))
     #   kernl_tensor_merged_summary_op=tf.summary.merge_all(scope="kernl_tensor_summaries")
    
    # init = tf.global_variables_initializer()
    # saver = tf.train.Saver()

In [10]:
kernl_outputs.Theta

<tf.Tensor 'rnn/transpose_5:0' shape=(1, 50, 2) dtype=float32>

In [12]:
tf.einsum('unv,',kernl_outputs.Theta,trainables[kernl_sensitivity_tensor_index])

ValueError: Shape must be rank 2 but is rank 3 for 'MatMul' (op: 'MatMul') with input shapes: [1,50,2], [2,2].